### Deep Learning for Computer Vision  
### Multi-Task Regression with the Digital Typhoon Dataset

This notebook demonstrates a **supervised multi-task regression** workflow using the **Digital Typhoon** dataset, which provides infrared (IR) satellite images of tropical cyclones paired with meteorological measurements.  

- **Dataset:** Digital Typhoon ([TorchGeo documentation](https://torchgeo.readthedocs.io/en/stable/api/datasets.html#digital-typhoon))  
- **Learning Paradigm:** Supervised learning  
- **Task Type:** Multi-task regression (scalar outputs per image)  
- **Input:**  
  - Single infrared (IR) satellite image    
- **Output / Target Variables:**  
  - Central wind speed (km/h)  
  - Minimum central pressure (hPa)

These targets are estimated directly from image appearance, making this a representative **vision-based regression problem** rather than a classification task.

**References**  
Digital Typhoon Dataset: *A Large-Scale Benchmark for Tropical Cyclone Analysis*      [arXiv:2411.16421](https://arxiv.org/pdf/2411.16421) ; [arXiv:2311.02665](https://arxiv.org/pdf/2311.02665)
